In [14]:
import feature_engineering as fe
import my_globals
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
import os
tf.enable_eager_execution()
tf.enable_v2_behavior()
# tf.disable_eager_execution()

from preprocessing import cleaning, preprocess_pipeline
from utils import get_sub_dataset

from gensim.models import Word2Vec

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import mixed_precision

#### Disabling GPU ####
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
####


params = dict(
    N = 100000,
    random_seed = 0,
    max_features = 5000,
    sequence_length = 300,
    batch_size = 256, 
    epochs = 30, 
    train = False
)

policy = mixed_precision.Policy(
    'mixed_float16'
)
mixed_precision.set_global_policy(policy)



num_threads = os.environ.get('OMP_NUM_THREADS')
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1)

if num_threads:
    session = tf.Session(config=tf.ConfigProto(
    gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
else:
    session = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

config = tf.ConfigProto(device_count = {'GPU': 0})
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.1
session = tf.Session(config=config)
K.set_session(session)


In [8]:
data = get_sub_dataset(size = params["N"], random_seed=params["random_seed"])
data = cleaning(data)
data["processed_text"] = data["text"].apply(
    lambda s:
    preprocess_pipeline(
        s,
        pipeline="w2v"
    )
)

if params["train"]:
    feature_df, w2v_model = fe.word_embedding(
        data,
        vector_size=params["max_features"],
        w2v_epochs=30,
        aggregate="l3",
        colname="processed_text"
    )
else:
    w2v_model = Word2Vec.load("../project_data/w2v_model.pickle")

C:\Users\linru\AppData\Roaming\Python\Python39\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [10]:
# Split

XX = data[["processed_text"]]
yy = data[["target"]]

df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(
    XX, yy, test_size=0.15, random_state = params["random_seed"]
)



In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_X_train["processed_text"])
vocab_size = len(tokenizer.word_index) + 1

X_train = pad_sequences(
    tokenizer.texts_to_sequences(df_X_train["processed_text"]), 
    maxlen=params["sequence_length"]
)
X_test = pad_sequences(
    tokenizer.texts_to_sequences(df_X_test["processed_text"]), 
    maxlen=params["sequence_length"]
)

encoder = LabelEncoder()
encoder.fit(df_y_train["target"].tolist())
y_train = encoder.transform(df_y_train["target"].tolist())
y_test = encoder.transform(df_y_test["target"].tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [12]:
# Construct embedding layer

embedding_matrix = np.zeros((vocab_size, params["max_features"]))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

with tf.device('cpu:0'):
  embedding_layer = Embedding(
    vocab_size, 
    params["max_features"],
    weights=[embedding_matrix], 
    input_length=params["sequence_length"], 
    trainable=False
  )
  embedding_layer.build(input_shape=params["sequence_length"])

(46715, 5000)


In [13]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
    EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [14]:
history = model.fit(X_train, y_train,
                    batch_size=params["batch_size"],
                    epochs=params["epochs"],
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Train on 76500 samples, validate on 8500 samples
Epoch 1/30


ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[5000,100] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node lstm/while/lstm_cell/split}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[metrics/acc/Identity/_87]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[5000,100] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node lstm/while/lstm_cell/split}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to